# 飞书机器人上手教程 (4) — 交互卡片

本 Notebook 演示飞书交互卡片（Interactive Card）的构建与回调处理。

## 卡片能做什么

- 展示格式化信息（表格、多列布局、Markdown）
- 嵌入按钮、下拉框、输入框等控件
- 用户点击按钮后，机器人收到回调并可**动态更新**卡片内容
- 弹出 Toast 提示

## 前置条件

- 飞书凭证：`FEISHU_APP_ID` / `FEISHU_APP_SECRET`  
- 如需测试卡片回调，需要启动机器人（WebSocket 模式）

In [2]:
from chattool.tools.lark import LarkBot

bot = LarkBot()
TEST_USER = "rexwzh"
ID_TYPE   = "user_id"

## 1. 基础卡片结构

一张飞书卡片由 `config`（配置）、`header`（标题栏）、`elements`（内容块）三部分组成。

In [3]:
# 最简单的卡片：只有标题和文字
simple_card = {
    "config": {"wide_screen_mode": True},
    "header": {
        "title": {"tag": "plain_text", "content": "📋 这是一张卡片"},
        "template": "blue",  # blue / red / yellow / green / grey / purple
    },
    "elements": [
        {
            "tag": "div",
            "text": {
                "tag": "lark_md",
                "content": "这是卡片正文，支持 **Markdown** 格式。\n\n可以换行、**加粗**、~~删除线~~ 等。",
            },
        }
    ],
}

resp = bot.send_card(TEST_USER, ID_TYPE, simple_card)
print("发送状态:", "✅" if resp.success() else f"❌ {resp.msg}")

发送状态: ✅


## 2. 常用卡片元素

### 2.1 分割线 + 多行文本

In [4]:
import time

info_card = {
    "config": {"wide_screen_mode": True},
    "header": {
        "title": {"tag": "plain_text", "content": "📊 今日数据汇报"},
        "template": "green",
    },
    "elements": [
        {
            "tag": "div",
            "text": {
                "tag": "lark_md",
                "content": f"**汇报时间**: {time.strftime('%Y-%m-%d %H:%M')}",
            },
        },
        {"tag": "hr"},  # 分割线
        {
            "tag": "div",
            "fields": [
                {"is_short": True, "text": {"tag": "lark_md", "content": "**新增用户**\n128 人"}},
                {"is_short": True, "text": {"tag": "lark_md", "content": "**活跃用户**\n856 人"}},
                {"is_short": True, "text": {"tag": "lark_md", "content": "**订单数**\n342 笔"}},
                {"is_short": True, "text": {"tag": "lark_md", "content": "**收入**\n¥ 28,600"}},
            ],
        },
        {"tag": "hr"},
        {
            "tag": "note",  # 备注
            "elements": [
                {"tag": "plain_text", "content": "数据来源：数据仓库，每小时更新"}
            ],
        },
    ],
}

resp = bot.send_card(TEST_USER, ID_TYPE, info_card)
print("发送状态:", "✅" if resp.success() else f"❌ {resp.msg}")

发送状态: ✅


### 2.2 带按钮的卡片

In [5]:
action_card = {
    "config": {"wide_screen_mode": True},
    "header": {
        "title": {"tag": "plain_text", "content": "📝 待办事项"},
        "template": "yellow",
    },
    "elements": [
        {
            "tag": "div",
            "text": {
                "tag": "lark_md",
                "content": "**任务**：完成 Q4 数据报告\n**截止**：今日 18:00\n**负责人**：Rex",
            },
        },
        {"tag": "hr"},
        {
            "tag": "action",
            "actions": [
                {
                    "tag": "button",
                    "text": {"tag": "plain_text", "content": "✅ 已完成"},
                    "type": "primary",
                    "value": {"action": "done", "task_id": "task_001"},
                },
                {
                    "tag": "button",
                    "text": {"tag": "plain_text", "content": "⏰ 延期"},
                    "type": "default",
                    "value": {"action": "postpone", "task_id": "task_001"},
                },
                {
                    "tag": "button",
                    "text": {"tag": "plain_text", "content": "❌ 取消"},
                    "type": "danger",
                    "value": {"action": "cancel", "task_id": "task_001"},
                },
            ],
        },
    ],
}

resp = bot.send_card(TEST_USER, ID_TYPE, action_card)
if resp.success():
    task_card_id = resp.data.message_id
    print(f"✅ 卡片发送成功，message_id = {task_card_id}")
else:
    print(f"❌ 发送失败: {resp.msg}")

✅ 卡片发送成功，message_id = om_x100b56c1cba4e0a0b2713c40f8c0f55


## 3. 卡片回调处理（@bot.card_action）

用户点击按钮后，飞书会向机器人发送回调事件。`@bot.card_action("action_value")` 装饰器根据 `value.action` 字段路由到对应处理函数。

In [6]:
import time

# 发送一张审批卡片
def build_approval_card(task_id: str, status: str = "pending"):
    if status == "pending":
        return {
            "config": {"wide_screen_mode": True},
            "header": {
                "title": {"tag": "plain_text", "content": "🔔 审批请求"},
                "template": "blue",
            },
            "elements": [
                {
                    "tag": "div",
                    "text": {
                        "tag": "lark_md",
                        "content": f"**申请人**: Rex\n**事由**: 请假一天\n**时间**: {time.strftime('%Y-%m-%d')}",
                    },
                },
                {"tag": "hr"},
                {
                    "tag": "action",
                    "actions": [
                        {
                            "tag": "button",
                            "text": {"tag": "plain_text", "content": "✅ 通过"},
                            "type": "primary",
                            "value": {"action": "approve", "task_id": task_id},
                        },
                        {
                            "tag": "button",
                            "text": {"tag": "plain_text", "content": "❌ 拒绝"},
                            "type": "danger",
                            "value": {"action": "reject", "task_id": task_id},
                        },
                    ],
                },
            ],
        }
    elif status == "approved":
        return {
            "config": {"wide_screen_mode": True},
            "header": {
                "title": {"tag": "plain_text", "content": "✅ 审批已通过"},
                "template": "green",
            },
            "elements": [
                {
                    "tag": "div",
                    "text": {
                        "tag": "lark_md",
                        "content": f"审批已于 `{time.strftime('%H:%M:%S')}` 通过。",
                    },
                }
            ],
        }
    else:  # rejected
        return {
            "config": {"wide_screen_mode": True},
            "header": {
                "title": {"tag": "plain_text", "content": "❌ 审批已拒绝"},
                "template": "red",
            },
            "elements": [
                {
                    "tag": "div",
                    "text": {
                        "tag": "lark_md",
                        "content": f"审批于 `{time.strftime('%H:%M:%S')}` 被拒绝。",
                    },
                }
            ],
        }


# ── 注册卡片回调处理器 ──────────────────────────────────
@bot.card_action("approve")
def on_approve(ctx):
    task_id = ctx.action_value.get("task_id")
    # 更新卡片为「已通过」状态
    ctx.update_card(build_approval_card(task_id, "approved"))
    # 弹出 Toast 提示
    ctx.toast("审批已通过 ✅", type="success")
    print(f"审批通过: task_id={task_id}, 操作者={ctx.operator_id}")

@bot.card_action("reject")
def on_reject(ctx):
    task_id = ctx.action_value.get("task_id")
    ctx.update_card(build_approval_card(task_id, "rejected"))
    ctx.toast("已拒绝", type="error")
    print(f"审批拒绝: task_id={task_id}")


# ── 发送审批卡片 ────────────────────────────────────────
resp = bot.send_card(TEST_USER, ID_TYPE, build_approval_card("task_demo_001"))
if resp.success():
    print(f"✅ 审批卡片已发送，message_id={resp.data.message_id}")
    print("点击卡片上的「通过」或「拒绝」按钮，卡片内容会自动更新")
    print("（需要启动机器人才能处理按钮回调）")
else:
    print(f"❌ 发送失败: {resp.msg}")

✅ 审批卡片已发送，message_id=om_x100b56c1cb6818a0b2e96cc1a05a85a
点击卡片上的「通过」或「拒绝」按钮，卡片内容会自动更新
（需要启动机器人才能处理按钮回调）


## 4. 更新已发送的卡片（主动 Patch）

除了在回调中更新，也可以主动调用 `patch` 接口更新卡片。

In [7]:
import lark_oapi as lark
from lark_oapi.api.im.v1 import PatchMessageRequest, PatchMessageRequestBody
import json, time

# 先发一张「进行中」的卡片
progress_card = {
    "config": {"wide_screen_mode": True},
    "header": {
        "title": {"tag": "plain_text", "content": "⏳ 任务处理中..."},
        "template": "grey",
    },
    "elements": [
        {"tag": "div", "text": {"tag": "lark_md", "content": "正在处理，请稍候..."}}
    ],
}

resp = bot.send_card(TEST_USER, ID_TYPE, progress_card)
if not resp.success():
    print(f"❌ 发送失败: {resp.msg}")
else:
    msg_id = resp.data.message_id
    print(f"✅ 发送「进行中」卡片，id={msg_id}")

    # 模拟 2 秒后任务完成
    time.sleep(2)

    # 更新为「完成」卡片
    done_card = {
        "config": {"wide_screen_mode": True},
        "header": {
            "title": {"tag": "plain_text", "content": "✅ 任务已完成"},
            "template": "green",
        },
        "elements": [
            {"tag": "div", "text": {"tag": "lark_md",
             "content": f"处理完成！耗时约 2 秒。\n完成时间：`{time.strftime('%H:%M:%S')}`"}}
        ],
    }

    patch_req = PatchMessageRequest.builder() \
        .message_id(msg_id) \
        .request_body(
            PatchMessageRequestBody.builder()
            .content(json.dumps(done_card))
            .build()
        ).build()

    patch_resp = bot.client.im.v1.message.patch(patch_req)
    print("更新状态:", "✅ 卡片已更新" if patch_resp.success() else f"❌ {patch_resp.msg}")

✅ 发送「进行中」卡片，id=om_x100b56c1c8c874a4b4cc7d714365153
更新状态: ✅ 卡片已更新


## 5. 卡片引用回复

In [8]:
# 先发一条普通文本
orig = bot.send_text(TEST_USER, ID_TYPE, "请看这张卡片报告 👇")
orig_id = orig.data.message_id

# 用卡片回复这条文本
summary_card = {
    "config": {"wide_screen_mode": True},
    "header": {"title": {"tag": "plain_text", "content": "📊 数据摘要"}, "template": "blue"},
    "elements": [
        {"tag": "div", "text": {"tag": "lark_md",
         "content": "| 指标 | 数值 |\n|------|------|\n| 今日 UV | 1,234 |\n| 今日 PV | 5,678 |\n| 转化率 | 3.2% |"}}
    ],
}

resp = bot.reply_card(orig_id, summary_card)
print("卡片引用回复:", "✅" if resp.success() else f"❌ {resp.msg}")

卡片引用回复: ✅


## 6. 启动机器人以处理卡片回调

In [10]:
# 启动机器人（后台 WebSocket，不阻塞 Notebook）
# 点击上面发送的审批卡片按钮，会在这里打印日志
bot.start_background()
print("🚀 机器人已启动，等待卡片按钮点击回调...")
print("点击刚才发送的审批卡片的「通过」或「拒绝」按钮")

🚀 机器人已启动，等待卡片按钮点击回调...
点击刚才发送的审批卡片的「通过」或「拒绝」按钮


## 小结

| 功能 | 方式 |
|------|------|
| 发送卡片 | `bot.send_card(user_id, id_type, card_dict)` |
| 卡片引用回复 | `bot.reply_card(message_id, card_dict)` |
| 处理按钮回调 | `@bot.card_action("action_key")` |
| 在回调中更新卡片 | `ctx.update_card(new_card)` |
| 弹出 Toast | `ctx.toast("消息", type="success/error/info")` |
| 主动更新已发卡片 | `client.im.v1.message.patch(PatchMessageRequest...)` |

### 卡片颜色主题

| `template` 值 | 颜色 | 常用场景 |
|---------------|------|----------|
| `blue` | 蓝色 | 通知、信息 |
| `green` | 绿色 | 成功、通过 |
| `yellow` | 黄色 | 警告、待处理 |
| `red` | 红色 | 错误、拒绝、告警 |
| `grey` | 灰色 | 进行中、禁用 |
| `purple` | 紫色 | 特殊标记 |

---

至此，四个 Notebook 教程已完成！

| 教程 | 内容 |
|------|------|
| [01 — 快速开始](lark_01_quickstart.ipynb) | 初始化、发送消息、引用回复 |
| [02 — 接收消息](lark_02_receiving.ipynb) | WebSocket 接收、路由装饰器 |
| [03 — AI 对话](lark_03_ai_chat.ipynb) | ChatSession、LLM 集成、多用户会话 |
| [04 — 交互卡片](lark_04_cards.ipynb) | 卡片构建、按钮回调、卡片更新 |